# Evaluation with `justcause`

In this notebook, we examplify how to use `justcause` in order to evaluate methods using reference datasets. For simplicity, we only use one dataset, but show how evaluation works with multiple methods. Both standard causal methods implemented in the framework as well as custom methods. 


## Custom First
The goal of the `justcause` framework is to be a modular and flexible facilitator of causal evaluation.

In [11]:
%load_ext autoreload

%autoreload 2

# Loading all required packages 
import itertools
import numpy as np
from sklearn.model_selection import train_test_split
from justcause.data import Col
from justcause.data.sets import load_ihdp
from justcause.metrics import pehe_score, mean_absolute
from justcause.evaluation import setup_result_df, setup_scores_df, calc_scores, \
    summarize_scores

from sklearn.linear_model import LinearRegression

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Setup data and methods you want to evaluate
Let's say we wanted to compare a S-Learner with propensity weighting, based on a propensity estimate of our choice. Thus, we cannot simply use the predefined SLearner from `justcause.learners`, but have to provide our own adaption, which first estimates propensities and uses these for fitting an adjusted model. 

By providing a "blackbox" method like below, you can choose to do whatever you want inside. For example, you can replace your predictions available factual outcomes, estimate the propensity in different ways or even use a true propensity, in case of a generated dataset, where it is available. You can also resort to out-of-sample prediction, where no information about treatment is provided to the method. 

In [12]:
from justcause.learners import SLearner
from justcause.learners.propensity import estimate_propensities

# Get the first 100 replications
replications = load_ihdp(select_rep=np.arange(100))
metrics = [pehe_score, mean_absolute]

train_size = 0.8
random_state = 42

def weighted_slearner(train, test):
    """
    Custom method that takes 'train' and 'test' CausalFrames (see causal_frames.ipynb)
    and returns ITE predictions for both after training on 'train'. 
    
    Implement your own method in a similar fashion to evaluate them within the framework!
    """
    train_X, train_t, train_y = train.np.X, train.np.t, train.np.y
    test_X, test_t, test_y = test.np.X, test.np.t, test.np.y
    
    
    # Get calibrated propensity estimates
    p = estimate_propensities(train_X, train_t)

    # Make sure the supplied learner is able to use `sample_weights` in the fit() method
    slearner = SLearner(LinearRegression())
    
    # Weight with inverse probability of treatment (inverse propensity)
    slearner.fit(train_X, train_t, train_y, weights=1/p)
    return (
        slearner.predict_ite(train_X, train_t, train_y),
        slearner.predict_ite(test_X, test_t, test_y)
    )

### Example Evaluation Loop
Now given a callable like `weighted_slearner` we can evaluate that method using multiple metrics on the given replications. 
The result dataframe then contains two rows with the summarized scores over all replications for train and test separately. 

In [13]:
results_df = setup_result_df(metrics)
    
test_scores = setup_scores_df(metrics)
train_scores = setup_scores_df(metrics)

for rep in replications:

    train, test = train_test_split(
        rep, train_size=train_size, random_state=random_state
    )

    # REPLACE this with the function you implemented and want to evaluate
    train_ite, test_ite = weighted_slearner(train, test)

    # Calculate the scores and append them to a dataframe
    test_scores = test_scores.append(calc_scores(
        test[Col.ite], test_ite, metrics
    ), ignore_index=True)

    train_scores = train_scores.append(calc_scores(
        train[Col.ite], train_ite, metrics
    ), ignore_index=True)

# Summarize the scores and save them in a dataframe
train_result, test_result = summarize_scores(train_scores), summarize_scores(test_scores)
train_result.update({'method': 'weighted_slearner', 'train': True})
test_result.update({'method': 'weighted_slearner', 'train': False})

results_df = results_df.append(train_result, ignore_index=True)
results_df = results_df.append(test_result, ignore_index=True)

In [14]:
results_df

,method,train,pehe_score-mean,pehe_score-median,pehe_score-std,mean_absolute-mean,mean_absolute-median,mean_absolute-std
0,weighted_slearner,True,5.592356,2.569472,8.248291,0.369939,0.212427,0.524395
1,weighted_slearner,False,5.493401,2.589651,7.903174,0.655602,0.287201,0.941941


Now in this case, using `justcause` has hardly any advantages, because only one dataset and one method is used. You might as well just implement all the evaluation manually. However, this can simply be expanded to more methods by looping over the callables.

In [15]:
def basic_slearner(train, test):
    """ """
    train_X, train_t, train_y = train.np.X, train.np.t, train.np.y
    test_X, test_t, test_y = test.np.X, test.np.t, test.np.y

    slearner = SLearner(LinearRegression())
    slearner.fit(train_X, train_t, train_y)
    return (
        slearner.predict_ite(train_X, train_t, train_y),
        slearner.predict_ite(test_X, test_t, test_y)
    )

methods = [basic_slearner, weighted_slearner]

results_df = setup_result_df(metrics)

for method in methods:
    
    test_scores = setup_scores_df(metrics)
    train_scores = setup_scores_df(metrics)

    for rep in replications:

        train, test = train_test_split(
            rep, train_size=train_size, random_state=random_state
        )

        # REPLACE this with the function you implemented and want to evaluate
        train_ite, test_ite = method(train, test)

        # Calculate the scores and append them to a dataframe
        test_scores = test_scores.append(calc_scores(
            test[Col.ite], test_ite, metrics
        ), ignore_index=True)

        train_scores = train_scores.append(calc_scores(
            train[Col.ite], train_ite, metrics
        ), ignore_index=True)

    # Summarize the scores and save them in a dataframe
    train_result, test_result = summarize_scores(train_scores), summarize_scores(test_scores)
    train_result.update({'method': method.__name__, 'train': True})
    test_result.update({'method': method.__name__, 'train': False})

    results_df = results_df.append(train_result, ignore_index=True)
    results_df = results_df.append(test_result, ignore_index=True)

In [16]:
results_df

,method,train,pehe_score-mean,pehe_score-median,pehe_score-std,mean_absolute-mean,mean_absolute-median,mean_absolute-std
0,weighted_slearner,True,5.633660,2.623297,8.362125,0.732443,0.238185,1.493276
1,weighted_slearner,False,5.625971,2.635993,8.213626,1.292668,0.396246,2.474603
2,weighted_slearner,True,5.592356,2.569472,8.248291,0.369939,0.212427,0.524395
3,weighted_slearner,False,5.493401,2.589651,7.903174,0.655602,0.287201,0.941941


And because in most cases, we're not changing anything within this loop for the ITE case, `justcause` provides a default implementation. 

## Standard Evaluation of ITE predictions
Using the same list of method callables, we can just call `evaluate_ite` and pass all the information. The default implementation sets up a dataframe for the result following a certain convention. 

First, there's two columns to define the method for which the results are as well as whether they've been calculated on train or test. Then for all supplied `metrics`, all `formats` will be listed. 

Standard `metrics` like (PEHE or Mean absolute error) are implemented in `justcause.metrics`. 
Standard formats used for summarizing the scores over multiple replications are `np.mean, np.median, np.std`, other possibly interesting formats could be *skewness*, *minmax*, *kurtosis*. A method provided as format must take an `axis` parameter, ensuring that it can be applied to the scores dataframe. 



In [19]:
from justcause.evaluation import evaluate_ite

result = evaluate_ite(replications, methods, metrics, train_size=train_size, random_state=random_state)

In [20]:
result

,method,train,pehe_score-mean,pehe_score-median,pehe_score-std,mean_absolute-mean,mean_absolute-median,mean_absolute-std
0,basic_slearner,True,5.633660,2.623297,8.362125,0.732443,0.238185,1.493276
1,basic_slearner,False,5.633660,2.623297,8.362125,0.732443,0.238185,1.493276
2,weighted_slearner,True,5.592356,2.569472,8.248291,0.369939,0.212427,0.524395
3,weighted_slearner,False,5.592356,2.569472,8.248291,0.369939,0.212427,0.524395


### Adding standard causal methods to the mix
Within `justcause.learners` we've implemented a couple of standard methods that provide a `predict_ite()` method. Instead of going the tedious way like we've done in `weighted_slearner` above, we can just use these methods directly. The default implementation will use a default base learner for all the meta-learners, fit the method on train and predict the ITEs for train and test. 

By doing so, we can get rid of the `basic_slearner` method above, because it just uses the default setting and procedure for fitting the model. Instead, we just use `SLearner(LinearRegression())`. 

In [21]:
from justcause.learners import TLearner, XLearner, RLearner

# All in standard configuration
methods = [SLearner(LinearRegression()), weighted_slearner, TLearner(), XLearner(), RLearner(LinearRegression())]

result = evaluate_ite(replications, methods, metrics, train_size=train_size, random_state=random_state)

In [22]:
result

,method,train,pehe_score-mean,pehe_score-median,pehe_score-std,mean_absolute-mean,mean_absolute-median,mean_absolute-std
0,SLearner(learner=LinearRegression),True,5.633660,2.623297,8.362125,0.732443,0.238185,1.493276
1,SLearner(learner=LinearRegression),False,5.633660,2.623297,8.362125,0.732443,0.238185,1.493276
2,weighted_slearner,True,5.592356,2.569472,8.248291,0.369939,0.212427,0.524395
3,weighted_slearner,False,5.592356,2.569472,8.248291,0.369939,0.212427,0.524395
4,"TLearner(control=LassoLars, treated=LassoLars)",True,5.572626,2.543798,8.213573,0.293187,0.166370,0.428028
5,"TLearner(control=LassoLars, treated=LassoLars)",False,5.572626,2.543798,8.213573,0.293187,0.166370,0.428028
6,"XLearner(outcome_c=LassoLars, outcome_t=LassoL...",True,5.579285,2.543798,8.240606,0.289699,0.166370,0.427008
7,"XLearner(outcome_c=LassoLars, outcome_t=LassoL...",False,5.579285,2.543798,8.240606,0.289699,0.166370,0.427008
8,"RLearner(outcome=LinearRegression, effect=Line...",True,2.560234,1.221982,3.731162,0.253945,0.152045,0.283504
9,"RLearner(outcome=LinearRegression, effect=Line...",False,2.560234,1.221982,3.731162,0.253945,0.152045,0.283504
